<a href="https://colab.research.google.com/github/klaudiajsk/machine-learning-I/blob/main/linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import sklearn

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

#permission
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d mirichoi0218/insurance

In [ ]:
from zipfile import ZipFile
file_name = 'insurance.zip'

with ZipFile (file_name, 'r') as zip:
  zip.extractall()
  print('Done')


In [ ]:
dataset = pd.read_csv('insurance.csv')
dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.duplicated().sum()

In [ ]:
dataset = dataset.drop_duplicates()
dataset.info()

In [ ]:
dataset_columns = [column for column in dataset.columns if dataset[column].dtype == 'O']

In [ ]:
for column in dataset_columns:
  dataset[column] = dataset[column].astype('category')
dataset.info()

In [ ]:
df_dummies = pd.get_dummies(dataset, drop_first=True)
df_dummies

In [ ]:
df_dummies.corr()['charges'].sort_values(ascending=False)

In [ ]:
data = df_dummies.copy()
target = data.pop('charges')
data.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target)

In [ ]:
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

In [ ]:
from sklearn.linear_model import LinearRegression
regression = LinearRegression()
regression.fit(X_train, y_train)
print(f'R2 score: {regression.score(X_test, y_test):.2f}')